In [2]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz  
!tar xf $SPARK_VERSION-bin-hadoop3.tgz 
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataFrameBasics").getOrCreate()

In [4]:
from pathlib import Path
import pandas as pd
from collections import Counter
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn import tree
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix

In [6]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://loanpredictions-bucket.s3.amazonaws.com/Loans_Data_ML.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("Loans_Data_ML.csv"), sep=",", header=True)

In [ ]:
df.show()

In [9]:
#Load Dataframe
data = Path('/content/Loans_Data_ML.csv')
df = pd.read_csv(data)
df.head()

,Loan_Amt,Loan_Term,Monthly_Payment,Credit_Grade,Employement_Length,Annual_Income,DTI_Ratio,Application_Type,Joint_Annual_Income,Joint_DTI_Ratio,...,Delinquent_Amount,#_Mortgage_Accounts,#_Bankruptcies,#_Tax_Liens,Balance_Exc_Mortgage,Joint_Revolving_Balance,Open_Acc_IL24m,Open_RevAcc_24m,Acc_Curr_PD30days,Loan_Status
0,3600,36,111.97,A4,10+ years,120000.0,18.90,INDIVIDUAL,0.0,0.0,...,0,0,1,0,36506,0.0,14,11,0.0,Issued
1,15000,60,356.78,C4,10+ years,125000.0,17.25,INDIVIDUAL,0.0,0.0,...,0,2,0,0,69364,0.0,3,3,0.0,Issued
2,8400,36,276.56,B3,8 years,50000.0,15.63,INDIVIDUAL,0.0,0.0,...,0,4,0,0,51591,0.0,5,12,0.0,Issued
3,4000,36,130.00,B2,2 years,50000.0,33.61,INDIVIDUAL,0.0,0.0,...,0,6,0,0,136208,0.0,7,4,0.0,Issued
4,6000,36,185.93,A3,3 years,125000.0,9.25,INDIVIDUAL,0.0,0.0,...,0,0,0,0,60622,0.0,15,3,0.0,Current


In [10]:
# Remove the `Issued` loan status
issued_mask = df['Loan_Status'] != 'Issued'
df = df.loc[issued_mask]
issued_mask = df['Loan_Status'] != 'Fully Paid'
df = df.loc[issued_mask]

# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,Loan_Amt,Loan_Term,Monthly_Payment,Credit_Grade,Employement_Length,Annual_Income,DTI_Ratio,Application_Type,Joint_Annual_Income,Joint_DTI_Ratio,...,Delinquent_Amount,#_Mortgage_Accounts,#_Bankruptcies,#_Tax_Liens,Balance_Exc_Mortgage,Joint_Revolving_Balance,Open_Acc_IL24m,Open_RevAcc_24m,Acc_Curr_PD30days,Loan_Status
0,6000,36,185.93,A3,3 years,125000.0,9.25,INDIVIDUAL,0.0,0.0,...,0,0,0,0,60622,0.0,15,3,0.0,low_risk
1,12000,36,370.48,A2,10+ years,56000.0,17.90,INDIVIDUAL,0.0,0.0,...,0,5,0,0,21540,0.0,5,0,0.0,low_risk
2,10500,60,345.15,G2,< 1 year,45000.0,17.25,INDIVIDUAL,0.0,0.0,...,0,0,1,0,22148,0.0,7,13,0.0,low_risk
3,18000,36,542.07,A1,< 1 year,155000.0,9.08,INDIVIDUAL,0.0,0.0,...,0,0,0,0,61257,0.0,4,1,0.0,low_risk
4,1600,36,53.72,C1,1 year,60000.0,12.62,INDIVIDUAL,0.0,0.0,...,0,0,0,0,27231,0.0,6,10,0.0,low_risk


In [11]:
# Create our features

X = df.drop(columns='Loan_Status', axis=1)
X = pd.get_dummies(X)

# Create our target
y = df["Loan_Status"]

In [12]:
# Check the balance of our target values
y.value_counts()

low_risk     78897
high_risk      560
Name: Loan_Status, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
X_train.shape

(59592, 72)

In [14]:
pip install imblearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
## Naive Random Oversampling

In [16]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'low_risk': 59172, 'high_risk': 59172})

In [17]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(random_state=1)

In [18]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.5496451204055767

In [19]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[   98,    42],
       [11849,  7876]])

In [20]:
# Print the imbalanced classification report
# Naive Random Oversampling
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.70      0.40      0.02      0.53      0.29       140
   low_risk       0.99      0.40      0.70      0.57      0.53      0.27     19725

avg / total       0.99      0.40      0.70      0.57      0.53      0.27     19865



In [21]:
## SMOTE Oversampling

In [22]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'low_risk': 59172, 'high_risk': 59172})

In [23]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(random_state=1)

In [24]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.5656961796125295

In [25]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   96,    44],
       [10934,  8791]])

In [26]:
# Print the imbalanced classification report
# SMOTE Oversampling
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.69      0.45      0.02      0.55      0.31       140
   low_risk       1.00      0.45      0.69      0.62      0.55      0.30     19725

avg / total       0.99      0.45      0.68      0.61      0.55      0.30     19865



In [27]:
## Undersampling

In [28]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=78)
model.fit(X_resampled, y_resampled)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(random_state=78)

In [29]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.5656961796125295

In [30]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   96,    44],
       [10934,  8791]])

In [ ]:
# Print the imbalanced classification report
# Undersampling
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

In [33]:
## Combination (Over and Under) Sampling

In [32]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(random_state=1)

In [34]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.5656961796125295

In [35]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   96,    44],
       [10934,  8791]])

In [36]:
# Print the imbalanced classification report
# Combination (Over and Under) Sampling
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.69      0.45      0.02      0.55      0.31       140
   low_risk       1.00      0.45      0.69      0.62      0.55      0.30     19725

avg / total       0.99      0.45      0.68      0.61      0.55      0.30     19865

